In [108]:
# spec: https://tools.ietf.org/html/rfc7519
# wiki: https://en.wikipedia.org/wiki/JSON_Web_Token
# hmac: https://pymotw.com/2/hmac/

import time
import datetime
import base64
import json

 
HOURS = 24

DOT = '.'
sr = "mysecret"
alg = 'HS256'

header = {"alg": "HS256", "typ": "JWT"}
payload = {"loggedInAs": "Manager", \
            "exp": str(datetime.datetime.utcnow()) + str(datetime.timedelta(seconds= 3600 * HOURS)),\
            "iat":   time.mktime(datetime.datetime.utcnow().timetuple())  }
print (json.dumps(header))
print (json.dumps(header), json.dumps(payload))


{"alg": "HS256", "typ": "JWT"}
('{"alg": "HS256", "typ": "JWT"}', '{"loggedInAs": "Manager", "exp": "2017-01-16 19:13:08.4407601 day, 0:00:00", "iat": 1484611988.0}')


### Unsigned token

In [109]:
unsigned_token = base64.b64encode(json.dumps(header) ) + DOT + base64.b64encode(json.dumps(payload) )
print (unsigned_token)


eyJhbGciOiAiSFMyNTYiLCAidHlwIjogIkpXVCJ9.eyJsb2dnZWRJbkFzIjogIk1hbmFnZXIiLCAiZXhwIjogIjIwMTctMDEtMTYgMTk6MTM6MDguNDQwNzYwMSBkYXksIDA6MDA6MDAiLCAiaWF0IjogMTQ4NDYxMTk4OC4wfQ==


### Signature wusing HMAC-SHA256

In [120]:
import hashlib
import hmac

print ("Hex Signature:" + hmac.new(sr, unsigned_token, digestmod=hashlib.sha256).hexdigest())

signature =  hmac.new(sr, unsigned_token, digestmod=hashlib.sha256).digest()
print ("Encoded Signaute: " +  base64.b64encode(signature))

Hex Signature:2e93d1807d9c5c7365fd508d9ba42e031658bd3b61760c22bf97e559e9ffc399
Encoded Signaute: LpPRgH2cXHNl/VCNm6QuAxZYvTthdgwiv5flWen/w5k=


### Sign token with signature

In [111]:
signed_token = unsigned_token +  DOT + base64.b64encode(signature)
print (signed_token)
# check it at https://jwt.io/

eyJhbGciOiAiSFMyNTYiLCAidHlwIjogIkpXVCJ9.eyJsb2dnZWRJbkFzIjogIk1hbmFnZXIiLCAiZXhwIjogIjIwMTctMDEtMTYgMTk6MTM6MDguNDQwNzYwMSBkYXksIDA6MDA6MDAiLCAiaWF0IjogMTQ4NDYxMTk4OC4wfQ==.LpPRgH2cXHNl/VCNm6QuAxZYvTthdgwiv5flWen/w5k=


### Check it back

In [121]:
h, p, s = signed_token.split('.')
print ("header:" + base64.b64decode(h))

header:{"alg": "HS256", "typ": "JWT"}


In [125]:
print ("payload:"   + base64.b64decode(p))
print ("signature:" + base64.b64decode(s))

payload:{"loggedInAs": "Manager", "exp": "2017-01-16 19:13:08.4407601 day, 0:00:00", "iat": 1484611988.0}
signature:.�р}�\se�P���.X�;av"���Y��Ù


In [126]:
ut = h + DOT + p
print ("unsighed token:" + ut)


unsighed token:eyJhbGciOiAiSFMyNTYiLCAidHlwIjogIkpXVCJ9.eyJsb2dnZWRJbkFzIjogIk1hbmFnZXIiLCAiZXhwIjogIjIwMTctMDEtMTYgMTk6MTM6MDguNDQwNzYwMSBkYXksIDA6MDA6MDAiLCAiaWF0IjogMTQ4NDYxMTk4OC4wfQ==


#### compare hashes

In [116]:
got_signature = base64.b64encode ( hmac.new(sr, ut, digestmod=hashlib.sha256).digest())
print (got_signature, base64.b64encode(base64.b64decode(s) ))

('LpPRgH2cXHNl/VCNm6QuAxZYvTthdgwiv5flWen/w5k=', 'LpPRgH2cXHNl/VCNm6QuAxZYvTthdgwiv5flWen/w5k=')


In [131]:
valid = got_signature == base64.b64encode(base64.b64decode(s)) 
print ("Valid?:" + str(valid))

Valid?:True
